# I have no clue what we're supposed to do

In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.util import bigrams
nltk.download('punkt')
nltk.download('wordnet')
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cacac\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cacac\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
reviews_as_table = pd.read_csv('IMDB Dataset.csv')
reviews_as_table

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


# cleaning

In [8]:
reviews_as_table["cleaned"] = reviews_as_table["review"].str.lower()
reviews_as_table.head()

,review,sentiment,cleaned
0,One of the other reviewers has mentioned that ...,positive,one of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,positive,a wonderful little production. <br /><br />the...
2,I thought this was a wonderful way to spend ti...,positive,i thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,negative,basically there's a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"petter mattei's ""love in the time of money"" is..."


In [9]:
reviews_as_table["cleaned"] = reviews_as_table["cleaned"].apply(lambda x: re.sub(r'[^\x00-\x7f]', r'', x))

reviews_as_table["cleaned"] = reviews_as_table["cleaned"].apply(lambda x: x.replace(".<br /><br />",".<br /><br /><s>"))
reviews_as_table["cleaned"] = reviews_as_table["cleaned"].apply(lambda x: x.replace(". <br /><br />",". <br /><br /><s>"))
reviews_as_table["cleaned"] = reviews_as_table["cleaned"].apply(lambda x: "<s> " + x)

reviews_as_table

,review,sentiment,cleaned
0,One of the other reviewers has mentioned that ...,positive,<s> one of the other reviewers has mentioned t...
1,A wonderful little production. <br /><br />The...,positive,<s> a wonderful little production. <br /><br /...
2,I thought this was a wonderful way to spend ti...,positive,<s> i thought this was a wonderful way to spen...
3,Basically there's a family where a little boy ...,negative,<s> basically there's a family where a little ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"<s> petter mattei's ""love in the time of money..."
...,...,...,...
49995,I thought this movie did a down right good job...,positive,<s> i thought this movie did a down right good...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,"<s> bad plot, bad dialogue, bad acting, idioti..."
49997,I am a Catholic taught in parochial elementary...,negative,<s> i am a catholic taught in parochial elemen...
49998,I'm going to have to disagree with the previou...,negative,<s> i'm going to have to disagree with the pre...


# counting

In [10]:
reviews_as_table["cleaned"] = reviews_as_table["cleaned"].apply(lambda x: word_tokenize(x))
reviews_as_table

,review,sentiment,cleaned
0,One of the other reviewers has mentioned that ...,positive,"[<, s, >, one, of, the, other, reviewers, has,..."
1,A wonderful little production. <br /><br />The...,positive,"[<, s, >, a, wonderful, little, production, .,..."
2,I thought this was a wonderful way to spend ti...,positive,"[<, s, >, i, thought, this, was, a, wonderful,..."
3,Basically there's a family where a little boy ...,negative,"[<, s, >, basically, there, 's, a, family, whe..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[<, s, >, petter, mattei, 's, ``, love, in, th..."
...,...,...,...
49995,I thought this movie did a down right good job...,positive,"[<, s, >, i, thought, this, movie, did, a, dow..."
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,"[<, s, >, bad, plot, ,, bad, dialogue, ,, bad,..."
49997,I am a Catholic taught in parochial elementary...,negative,"[<, s, >, i, am, a, catholic, taught, in, paro..."
49998,I'm going to have to disagree with the previou...,negative,"[<, s, >, i, 'm, going, to, have, to, disagree..."
